#Install libraries

In [1]:
!pip install deep_sort_realtime ultralytics


[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


#Import linraries

In [2]:
import torch
import cv2
import time
from ultralytics import YOLO

import time
import torch
import cv2
import torch.backends.cudnn as cudnn
from PIL import Image
import colorsys
import numpy as np
import keras
import os
import torch
from ultralytics import YOLO
import tensorflow as tf
from deep_sort_realtime.deepsort_tracker import DeepSort
from tensorflow.keras.models import load_model
import math

#Load model

In [4]:
# Load YOLOv8 model
# model = YOLO("yolov8x.pt")
model = YOLO("..\pose-based-method\yolov8n.pt")


In [5]:
#Load model
model_normal = load_model('./model_mobilenet2.keras')
model_normal.summary()

Model: "functional_36"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_14 (InputLayer)     │ (None, 448, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling_4 (Rescaling)         │ (None, 448, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 14, 7, 1280)    │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_4      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 512)            │       655,872 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,015,109 (19.13 MB)

 Trainable params: 919,041 (3.51 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

 Optimizer params: 1,838,084 (7.01 MB)

In [6]:
def predict_fall(img):
  global model_normal
  if img is None or img.size == 0:
    return False
  img_resized = cv2.resize(img, (224,448))
  img_rgb = cv2.cvtColor(img_resized, cv2.COLOR_BGR2RGB)
  img_array = np.array(img_rgb)

  img_array = np.expand_dims(img_array, axis=0)

  predictions = model_normal.predict(img_array,verbose = 0)
  return False if predictions[0][0] > 0.5 else True


In [7]:
import time

class ExpiringDict:
    def __init__(self, expiry_time=5):
        self.store = {}
        self.expiry_time = expiry_time

    def _remove_expired(self):
        current_time = time.time()
        keys_to_remove = [key for key, (value, timestamp) in self.store.items()
                          if current_time - timestamp > self.expiry_time]
        for key in keys_to_remove:
            del self.store[key]

    def __setitem__(self, key, value):
        current_time = time.time()
        self.store[key] = (value, current_time)
        self._remove_expired()

    def __getitem__(self, key):
        self._remove_expired()
        if key in self.store:
            value, _ = self.store[key]
            self.store[key] = (value, time.time())
            return value
        else:
            raise KeyError(f"Key '{key}' not found")

    def __delitem__(self, key):
        self._remove_expired()
        if key in self.store:
            del self.store[key]
        else:
            raise KeyError(f"Key '{key}' not found")

    def __contains__(self, key):
        self._remove_expired()
        if key in self.store:
            value, _ = self.store[key]
            self.store[key] = (value, time.time())
            return True
        return False

    def __repr__(self):
        self._remove_expired()
        return repr({key: value for key, (value, _) in self.store.items()})

In [14]:

cap = cv2.VideoCapture(r'C:\Users\NHAN\fall-detection\pose-based-method\data\videos\input\input5.mp4')
if not cap.isOpened():
    raise IOError("Cannot open video")

ed = ExpiringDict(expiry_time=60)
ed_sp = ExpiringDict(expiry_time=60)

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

tracker = DeepSort(max_age=int(fps),embedder="mobilenet")


fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(r'C:\Users\NHAN\fall-detection\pose-based-method\data\videos\output\output5_2.mp4', fourcc, fps, (width, height))
while True:
    ret, frame = cap.read()
    if not ret:
        break
    results = model(frame,verbose = False)

    detections = []
    dict_yolo = {}
    frame_copy = frame.copy()
    for result in results:
        boxes = result.boxes.xyxy.tolist()
        classes = result.boxes.cls.tolist()
        names = result.names
        confidences = result.boxes.conf.tolist()

        for box, cls, conf in zip(boxes, classes, confidences):
          x1, y1, x2, y2 = box
          w, h = x2 - x1, y2 - y1
          confidence = conf
          detected_class = cls
          name = names[int(cls)]
          if confidence > 0.5 and name == 'person':
            tuple_people = (x1, y1, x2, y2)
            dict_yolo[tuple_people] = 0
            detections.append(([x1, y1, w, h], confidence, detected_class))
    tracks = tracker.update_tracks(detections, frame=frame)

    for track in tracks:
        if not track.is_confirmed():
            continue
        track_id = track.track_id
        bbox = track.to_ltrb()  # Get bounding box
        x1, y1, x2, y2 = map(int, bbox)  # Convert coordinates to integers
        if y1 < y2 and x1 < x2:
          person_roi = frame[y1:y2,x1:x2]
        else:
          person_roi = None
        value_fall = 0
        if person_roi is not None:
          is_fall = predict_fall(person_roi)
          value_fall = 1 if is_fall else -1
        if track_id not in ed or track_id not in ed_sp:
            ed[track_id] = 0
            ed_sp[track_id] = 0
        ed[track_id] += value_fall
        ed_sp[track_id] +=1
        if ed[track_id] >= 0.2*fps:
          cv2.putText(frame_copy, f"FALL DETECTED ID: {ed[track_id]}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
          cv2.rectangle(frame_copy, (x1, y1), (x2, y2), (0, 0, 255), 2)
        else:
          cv2.rectangle(frame_copy, (x1, y1), (x2, y2), (0, 255, 0), 2)
          cv2.putText(frame_copy, f"ID: {ed[track_id]}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        if ed_sp[track_id] > fps:
          ed_sp[track_id] = 0
          if  0 < ed[track_id] < 2:
            ed[track_id] = 1
          elif 0 > ed[track_id] > -2:
            ed[track_id] = -1
          elif ed[track_id] >= 0.2*fps:
            ed[track_id] = int(0.2*fps) +1
          elif ed[track_id] > 2:
            ed[track_id] = int(min(fps*0.1,math.ceil(ed[track_id] / 2)))
          else:
            ed[track_id] = int(max(-1*fps*0.1, math.ceil(ed[track_id] / 2)))
    out.write(frame_copy)

cap.release()
out.release()
cv2.destroyAllWindows()